In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_398497/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_14_8_8,0.994433,0.965867,0.980017,0.984327,1.639062,2.656659,5.111375,3.811822,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1,model_14_8_9,0.994404,0.964601,0.977709,0.982971,1.647586,2.755214,5.701658,4.141773,"Hidden Size=[35], regularizer=0.1, learning_ra..."
2,model_14_8_7,0.994402,0.967113,0.982437,0.985737,1.648202,2.559658,4.492257,3.469114,"Hidden Size=[35], regularizer=0.1, learning_ra..."
3,model_14_8_10,0.994332,0.963343,0.975528,0.981679,1.668790,2.853102,6.259425,4.456077,"Hidden Size=[35], regularizer=0.1, learning_ra..."
4,model_14_8_6,0.994289,0.968303,0.984947,0.987180,1.681375,2.467063,3.850246,3.117974,"Hidden Size=[35], regularizer=0.1, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...
183,model_5_9_0,0.985049,0.975704,0.975813,0.975858,4.402021,4.296639,5.511576,4.868370,"Hidden Size=[15], regularizer=0.01, learning_r..."
191,model_16_9_0,0.984533,0.980771,0.994404,0.987671,4.554148,5.707980,1.913889,3.922515,"Hidden Size=[40], regularizer=0.1, learning_ra..."
195,model_7_8_2,0.984154,0.993796,0.957922,0.983836,4.665682,1.680401,6.954459,4.162313,"Hidden Size=[20], regularizer=0.01, learning_r..."
207,model_8_9_3,0.983640,0.982496,0.932278,0.968093,4.816861,4.582518,15.164958,9.562496,"Hidden Size=[20], regularizer=0.1, learning_ra..."
